In [2]:
pwd

'/dfs6/pub/gshalgum/CEE290AHW'

In [3]:
import numpy as np
from scipy.interpolate import interp1d

def interceptionModel(t, S, P, E0, a, b, c, d):
    """
    Single layer interception model.
    
    Parameters:
        t (float): current time [days]
        S (float): current storage [mm]
        P (numpy.ndarray): rainfall time series [[time, value]]
        E0 (numpy.ndarray): potential evaporation time series [[time, value]]
        a (float): interception efficiency [-]
        b (float): drainage parameter [1/d]
        c (float): maximum storage capacity [mm]
        d (float): evaporation efficiency [-]
    
    Returns:
        dSdt (float): change in storage with time [mm/day]
    """

    # Create interpolation functions for P and E0
    P_interp = interp1d(P[:, 0], P[:, 1], fill_value="extrapolate")
    E0_interp = interp1d(E0[:, 0], E0[:, 1], fill_value="extrapolate")

    # Interpolate to find the values at the current time
    P_int = max(P_interp(t), 0)
    E0_int = max(E0_interp(t), 0)

    # Calculate interception (unknown parameter x rainfall intensity)
    I = a * P_int

    # Calculate drainage (only if storage larger than storage capacity)
    D = b * (S - c) if S > c else 0

    # Calculate evaporation
    E = d * E0_int * (S / c) if c != 0 else 0

    # Calculate the change in storage
    dSdt = I - D - E

    return dSdt

/opt/apps/python/3.10.2/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
import scipy.io

# Load measurement data from a .mat file with a full path
data = scipy.io.loadmat('/dfs6/pub/gshalgum/CEE290AHW/measurement.mat')


# Extracting data from the loaded .mat file
obsTime = data['obsTime'].flatten()    # Make sure 'obsTime' is the correct key in the .mat file
obsPotEvap = data['obsPotEvap']        # Likewise for 'obsPotEvap'
obsPrecip = data['obsPrecip']          # And 'obsPrecip'
obsStorage = data['obsStorage'].flatten()  # And 'obsStorage'


In [9]:
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import scipy.io

import numpy as np
from scipy.interpolate import interp1d
from scipy.integrate import solve_ivp

def interceptionModel(t, S, P, E0, a, b, c, d):
    """
    Single layer interception model.
    
    Parameters:
        t (float): current time [days]
        S (float): current storage [mm]
        P (numpy.ndarray): rainfall time series [[time, value]]
        E0 (numpy.ndarray): potential evaporation time series [[time, value]]
        a (float): interception efficiency [-]
        b (float): drainage parameter [1/d]
        c (float): maximum storage capacity [mm]
        d (float): evaporation efficiency [-]
    
    Returns:
        dSdt (float): change in storage with time [mm/day]
    """
    # Create interpolation functions for P and E0
    P_interp = interp1d(P[:, 0], P[:, 1], bounds_error=False, fill_value="extrapolate")
    E0_interp = interp1d(E0[:, 0], E0[:, 1], bounds_error=False, fill_value="extrapolate")

    # Interpolate to find the values at the current time
    P_int = max(P_interp(t), 0)
    E0_int = max(E0_interp(t), 0)

    # Calculate interception
    I = a * P_int

    # Calculate drainage
    D = b * (S - c) if S > c else 0

    # Calculate evaporation
    E = np.nan_to_num(d * E0_int * (S / c)) if c != 0 else 0


    # Calculate the change in storage
    dSdt = I - D - E

    return np.array([dSdt])

def interceptionModel_OF(params, obsTime, obsPrecip, obsPotEvap, obsStorage):
    """
    Objective function for optimization: sum of squared differences between observed and modeled storage.
    """
    def model_rhs(t, S, P, E0, a, b, c, d):
        return interceptionModel(t, S, P, E0, a, b, c, d)

    # Wrap parameters P, E0, and model parameters a, b, c, d using a lambda in solve_ivp
    sol = solve_ivp(lambda t, S: model_rhs(t, S, obsPrecip, obsPotEvap, *params), 
                    [obsTime[0], obsTime[-1]], [obsStorage[0]], t_eval=obsTime)

    modeled_storage = sol.y.flatten()
    return np.sum((modeled_storage - obsStorage)**2)


# Load measurement data from a .mat file with a full path
data = scipy.io.loadmat('/dfs6/pub/gshalgum/CEE290AHW/measurement.mat')


# Extracting data from the loaded .mat file
obsTime = data['obsTime'].flatten()    # Make sure 'obsTime' is the correct key in the .mat file
obsPotEvap = data['obsPotEvap']        # Likewise for 'obsPotEvap'
obsPrecip = data['obsPrecip']          # And 'obsPrecip'
obsStorage = data['obsStorage'].flatten()  # And 'obsStorage'


# Number of trials for the Simplex method
n_Simplex_runs = 10

# Parameter ranges
ParRange = {
    'minn': np.array([0, 1, 0, 0]),
    'maxn': np.array([1, 1000, 5, 3])
}

# Optimization settings
options = {'disp': True, 'maxiter': 1000, 'maxfev': 1000}

# Initialize arrays for results
results = np.zeros((n_Simplex_runs, 4))
fvals = np.zeros(n_Simplex_runs)

# Run the optimization multiple times
for i in range(n_Simplex_runs):
    start_solution = ParRange['minn'] + np.random.rand(4) * (ParRange['maxn'] - ParRange['minn'])
    result = opt.minimize(interceptionModel_OF, start_solution, args=(obsTime, obsPrecip, obsPotEvap, obsStorage), method='Nelder-Mead', options=options)
    results[i] = result.x
    fvals[i] = result.fun

# Print the best result
print("Best set of parameters:", results[np.argmin(fvals)])
print("Minimum objective function value:", min(fvals))

# Plot results for visualization (if necessary)

Optimization terminated successfully.
         Current function value: 1.813377
         Iterations: 105
         Function evaluations: 226
Optimization terminated successfully.
         Current function value: 0.041237
         Iterations: 182
         Function evaluations: 437
Optimization terminated successfully.
         Current function value: 11.709086
         Iterations: 172
         Function evaluations: 358
Optimization terminated successfully.
         Current function value: 7.083130
         Iterations: 141
         Function evaluations: 312
Optimization terminated successfully.
         Current function value: 7.944172
         Iterations: 227
         Function evaluations: 433
Optimization terminated successfully.
         Current function value: 0.044157
         Iterations: 144
         Function evaluations: 322
Optimization terminated successfully.
         Current function value: 3.691184
         Iterations: 524
         Function evaluations: 948
Optimization termin

In [10]:
from scipy.optimize import minimize

# Define bounds for each parameter: [(min, max), (min, max), ...]
bounds = [(0, 1), (1, 1000), (0, 5), (0, 3)]

result = minimize(interceptionModel_OF, start_solution, args=(obsTime, obsPrecip, obsPotEvap, obsStorage),
                  method='L-BFGS-B', bounds=bounds, options={'disp': True})


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.61327D+00    |proj g|=  2.92545D+02

At iterate    1    f=  6.59892D+00    |proj g|=  7.06455D+02

At iterate    2    f=  6.58250D+00    |proj g|=  7.06455D+02
  ys=-9.338E-01  -gs= 6.509E-01 BFGS update SKIPPED



 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches



 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.


Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4      3     68     10     1     4   7.065D+02   6.582D+00
  F =   6.5824991450841317     

ABNORMAL_TERMINATION_IN_LNSRCH                              


In [13]:
# minimizing cost function added to SSR penalty term 
def interceptionModel_OF(params, obsTime, obsPrecip, obsPotEvap, obsStorage, lambda_reg):
    # Function to be integrated by solve_ivp
    def residuals(t, y, P, E0, a, b, c, d):
        return interceptionModel(t, y, P, E0, a, b, c, d)

    # Current parameter values and extra parameters need to be passed as a tuple
    extra_args = (obsPrecip, obsPotEvap, *params)

    # Calling solve_ivp with extra arguments
    sol = solve_ivp(residuals, [obsTime[0], obsTime[-1]], [obsStorage[0]], t_eval=obsTime, args=extra_args)
    modeled_storage = sol.y.flatten()
    ssr = np.sum((modeled_storage - obsStorage)**2)
    
    # Calculate L2 penalty term
    l2_penalty = lambda_reg * np.sum((params - np.array([0.5, 500.5, 2.5, 1.5]))**2)

    # Total cost is the sum of SSR and the regularization term
    return ssr + l2_penalty

# Example usage:
lambda_reg = 100  # Regularization strength, 
start_solution = [0.5, 500, 2.5, 1.5]  # Example initial parameter values
result = minimize(
    interceptionModel_OF, start_solution,
    args=(obsTime, obsPrecip, obsPotEvap, obsStorage, lambda_reg),
    method='L-BFGS-B', bounds=bounds, options={'disp': True}
)



RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.02009D+01    |proj g|=  4.99000D+02

At iterate    1    f=  3.01977D+01    |proj g|=  4.99000D+02

At iterate    2    f=  3.01629D+01    |proj g|=  5.00000D+02

At iterate    3    f=  3.01035D+01    |proj g|=  5.00000D+02

At iterate    4    f=  3.00832D+01    |proj g|=  4.99000D+02
  ys=-2.463E-01  -gs= 1.595E-01 BFGS update SKIPPED




   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


At iterate    5    f=  3.00832D+01    |proj g|=  4.99000D+02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4      5     53      8     1     0   4.990D+02   3.008D+01
  F =   30.083153598581717     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


In [14]:
from scipy.optimize import minimize
import numpy as np

# Assuming interceptionModel and other required functions and data are defined

def interceptionModel_OF(params, obsTime, obsPrecip, obsPotEvap, obsStorage, lambda_reg):
    # Function to be integrated by solve_ivp
    def residuals(t, y, P, E0, a, b, c, d):
        return interceptionModel(t, y, P, E0, a, b, c, d)

    # Current parameter values and extra parameters need to be passed as a tuple
    extra_args = (obsPrecip, obsPotEvap, *params)

    # Calling solve_ivp with extra arguments
    sol = solve_ivp(residuals, [obsTime[0], obsTime[-1]], [obsStorage[0]], t_eval=obsTime, args=extra_args)
    modeled_storage = sol.y.flatten()
    ssr = np.sum((modeled_storage - obsStorage)**2)
    
    # Calculate L2 penalty term
    l2_penalty = lambda_reg * np.sum((params - np.array([0.5, 500.5, 2.5, 1.5]))**2)

    # Total cost is the sum of SSR and the regularization term
    return ssr + l2_penalty

# Set regularization strength and initial parameters
lambda_reg = 100  
start_solution = [0.5, 500, 2.5, 1.5]  
bounds = [(0, 1), (1, 1000), (0, 5), (0, 3)]  # Define bounds for each parameter

# Perform optimization
result = minimize(
    interceptionModel_OF, start_solution,
    args=(obsTime, obsPrecip, obsPotEvap, obsStorage, lambda_reg),
    method='L-BFGS-B', bounds=bounds, options={'disp': True}
)

# Print the optimized parameters
print("Optimized Parameters:", result.x)

# Check if parameters are within the bounds
in_bounds = np.all([(low <= p <= high) for (low, high), p in zip(bounds, result.x)])
print("Parameters within the defined bounds:", in_bounds)

# Optionally, print whether each parameter is within its bounds
for (low, high), p in zip(bounds, result.x):
    print(f"Parameter {p} is within bounds ({low}, {high}):", low <= p <= high)


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.02009D+01    |proj g|=  4.99000D+02

At iterate    1    f=  3.01977D+01    |proj g|=  4.99000D+02

At iterate    2    f=  3.01629D+01    |proj g|=  5.00000D+02

At iterate    3    f=  3.01035D+01    |proj g|=  5.00000D+02

At iterate    4    f=  3.00832D+01    |proj g|=  4.99000D+02
  ys=-2.463E-01  -gs= 1.595E-01 BFGS update SKIPPED
Optimized Parameters:
At iterate    5    f=  3.00832D+01    |proj g|=  4.99000D+02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint


   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.
